Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [2]:
import os
import json
import shutil

from reco_utils.dataset.criteo import get_spark_schema, load_spark_df

from azureml.core import Workspace
from azureml.core import VERSION as azureml_version

from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.image import ContainerImage

# Check core SDK version number
print("Azure ML SDK version: {}".format(azureml_version))

Azure ML SDK version: 1.0.8

## Configure Scoring Service Variables

In [4]:
AML_CONFIG_PATH = "/dbfs/FileStore/aml_config/config.json"
IMAGE_NAME = 'lightgbm-criteo-base'
IMAGE_NAME = 'lightgbm-criteo-streaming'
IMAGE_NAME = 'lightgbm-criteo-streamselect'
# IMAGE_NAME = 'lightgbm-criteo-baseselect'

#configure ACI
myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Azure ML ACI for LightGBM', 'algorithm':'LightGBM', 'IMAGE_NAME': IMAGE_NAME}, 
    description = 'This is for a light gbm example.'
)


In [5]:
ws = Workspace.from_config(AML_CONFIG_PATH)

Found the config file in: /dbfs/FileStore/aml_config/config.json

## Grab the Image

In [7]:
## Get the image based on image name
images = ContainerImage.list(ws)
images = [i for i in images if i.name == IMAGE_NAME]
versions = [i.version for i in images]
image = [i for i in images if i.version == max(versions)][0]
print(image)

ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7f10f2cf5240>, name=lightgbm-criteo-streamselect, id=lightgbm-criteo-streamselect:3, tags={'model': 'criteo-lgbm.model', 'implementation': 'streamselect', 'runtime': 'spark-PY'}, properties={}, version=3)

## Create the Service

Just using ACI to simplify the debugging experience and make sure it's nothing to do with AKS

In [9]:
# Webservice creation
myservice = Webservice.deploy_from_image(
  workspace=ws, 
  name=IMAGE_NAME,
  image=image,
  deployment_config = myaci_config
    )

myservice.wait_for_deployment(show_output=True)

Creating service
Running................................................................................................................................................................................................................................
TimedOutACI service creation operation finished, operation "TimedOut"
Service creation polling reached terminal state, current service state: Unhealthy
{
 "code": "AciHealthEndpointNotUp",
 "message": "Health endpoint is not up"
}

## Test the Service

Next, we can use data from the `test` data to test the service.

The service expects JSON as its payload, so we take the test data, fill missing values, convert to JSON, then submit to the service endpoint.

We have to fill in missing values here to create the data, because the webservice expects that the data coming into the webservice is well-formed.

In [11]:
# View the URI
url = myservice.scoring_uri
print('ACI URI: {}'.format(url))

# Setup authentication using one of the keys from aks_service
# headers = dict(Authorization='Bearer {}'.format(myservice.get_keys()[0]))

ACI URI: http://52.241.128.36:80/score

In [12]:
# Grab some sample data - these are all missing
df = load_spark_df(size='sample', spark=spark, dbutils=dbutils)
data = df.head().asDict()
## these have to be filled for the base:
## data = df.fillna('M').fillna(0).head().asDict()
## json.dumps(data)

0.00B [00:00, ?B/s] 0%| | 0.00/8.79M [00:00<?, ?B/s] 1%| | 49.2k/8.79M [00:00<00:28, 306kB/s] 3%|▎ | 221k/8.79M [00:00<00:21, 390kB/s]  10%|█ | 893k/8.79M [00:00<00:14, 540kB/s] 20%|█▉ | 1.72M/8.79M [00:00<00:09, 747kB/s] 39%|███▉ | 3.45M/8.79M [00:00<00:05, 1.04MB/s] 59%|█████▊ | 5.14M/8.79M [00:01<00:02, 1.42MB/s] 78%|███████▊ | 6.84M/8.79M [00:01<00:01, 1.92MB/s] 96%|█████████▌| 8.40M/8.79M [00:01<00:00, 2.52MB/s]8.79MB [00:01, 6.36MB/s] 
 Out[ 105 ]: '{"int05": null, "cat17": null, "cat24": null, "cat13": null, "cat14": null, "cat11": null, "cat05": null, "cat03": null, "int01": null, "cat18": null, "int10": null, "cat21": null, "int00": null, "cat07": null, "cat00": null, "int12": null, "cat23": null, "cat12": null, "cat08": null, "int06": null, "int11": null, "cat15": null, "cat01": null, "cat19": null, "cat20": null, "int04": null, "cat02": null, "cat10": null, "int03": null, "cat25": null, "int09": null, "int08": null, "cat09": null, "label": null, "cat06": null, "cat16": null, "int07": null, "int02": null, "cat22": null, "cat04": null}'

In [13]:
n_samples_to_test = 1
test=spark.table('test')
test_for_service_df = test.drop('features').fillna('M').fillna(0).limit(n_samples_to_test)
# display(test_for_service_df)
data_for_request=test_for_service_df.toJSON().collect()
test_json_str = json.dumps(data_for_request)
print(data_for_request)


['{"label":0,"int00":0,"int01":-2,"int02":0,"int03":0,"int04":12911,"int05":15,"int06":36,"int07":6,"int08":20,"int09":0,"int10":4,"int11":0,"int12":0,"cat00":"05db9164","cat01":"ef69887a","cat02":"612ba6a7","cat03":"f99777e0","cat04":"25c83c98","cat05":"M","cat06":"06a7d0ff","cat07":"0b153874","cat08":"a73ee510","cat09":"50c56209","cat10":"52d28861","cat11":"78f834bc","cat12":"a4b04123","cat13":"b28479f6","cat14":"902a109f","cat15":"6fc3b6d0","cat16":"e5ba7672","cat17":"4bcc9449","cat18":"2ba39771","cat19":"a458ea53","cat20":"782c1229","cat21":"M","cat22":"32c7478e","cat23":"482f757d","cat24":"47907db5","cat25":"b3834924"}']

In [14]:
test_json_str

Out[ 98 ]: '["{\\"label\\":0,\\"int00\\":0,\\"int01\\":-2,\\"int02\\":0,\\"int03\\":0,\\"int04\\":12911,\\"int05\\":15,\\"int06\\":36,\\"int07\\":6,\\"int08\\":20,\\"int09\\":0,\\"int10\\":4,\\"int11\\":0,\\"int12\\":0,\\"cat00\\":\\"05db9164\\",\\"cat01\\":\\"ef69887a\\",\\"cat02\\":\\"612ba6a7\\",\\"cat03\\":\\"f99777e0\\",\\"cat04\\":\\"25c83c98\\",\\"cat05\\":\\"M\\",\\"cat06\\":\\"06a7d0ff\\",\\"cat07\\":\\"0b153874\\",\\"cat08\\":\\"a73ee510\\",\\"cat09\\":\\"50c56209\\",\\"cat10\\":\\"52d28861\\",\\"cat11\\":\\"78f834bc\\",\\"cat12\\":\\"a4b04123\\",\\"cat13\\":\\"b28479f6\\",\\"cat14\\":\\"902a109f\\",\\"cat15\\":\\"6fc3b6d0\\",\\"cat16\\":\\"e5ba7672\\",\\"cat17\\":\\"4bcc9449\\",\\"cat18\\":\\"2ba39771\\",\\"cat19\\":\\"a458ea53\\",\\"cat20\\":\\"782c1229\\",\\"cat21\\":\\"M\\",\\"cat22\\":\\"32c7478e\\",\\"cat23\\":\\"482f757d\\",\\"cat24\\":\\"47907db5\\",\\"cat25\\":\\"b3834924\\"}"]'

In [15]:
## This needs to be a list for base...
## straight dict for streaming...
response = requests.post(url=url, json=[data])
print(response.json())

"cannot resolve '`cat00`' given input columns: [_corrupt_record];;\n'Project ['cat00]\n+- LogicalRDD [_corrupt_record#442], false\n"

### Delete the Service

When you are done, you can delete the service to minimize costs. You can always redeploy from the image using the same command above.

In [17]:
# Uncomment the following line to delete the web service
aks_service.delete()

In [18]:
aks_service.state